#***Ejercicio 4: Merton***

Utilizando información pública de la acción que ustedes decidan e información de su balance para obtener K0, calcular D, σK, para T = 1 año, y probabilidad de incumplimiento al tiempo T bajo el modelo de Merton. Calcular volatilidad anual de σA con la historia de acciones.

Paso de carga de librerías en R de todas las que se utilizarán

In [ ]:
install.packages("quantmod")
install.packages("curl")
install.packages("data.table")
install.packages("RCurl")
install.packages("bitops")
install.packages("setInternet2")
install.packages("rootSolve")
install.packages("xml2")
library(rootSolve)
library(quantmod)
library(RCurl)
library(data.table)
library(xml2)
require(jsonlite) #para el getQuotes


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘xts’, ‘zoo’, ‘TTR’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘bitops’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘setInternet2’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘li

In [ ]:
Symbols<-c  ("AMXL.MX")
length(Symbols)


start_date=Sys.Date()-3660 #fecha inicial

#Creación del objeto para guardar los datos
dataEnv<-new.env()

#obtener los datos
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date)


#limpiarlos, alinearnos y quedarnos con el precio de cierre 

#bt.prep(dataEnv,align='remove.na',fill.gaps = T)
#stock_prices = dataEnv$prices


#muestra de datos
#Nos quedamos con los precios
stock_prices = na.omit(dataEnv$`AMXL.MX`)
stock_prices=stock_prices[,4] #Close
tail(stock_prices[,])


[1] 1

[1] "AMXL.MX"

           AMXL.MX.Close
2022-01-21         19.77
2022-01-24         19.95
2022-01-25         19.83
2022-01-26         19.81
2022-01-27         19.03
2022-01-28         19.02

In [ ]:

#PARA CARGAR EL NÚMERO DE ACCIONES EN VENTA

getQuote <- function(ticks) {
  qRoot <- "https://query1.finance.yahoo.com/v7/finance/quote?fields=symbol,longName,regularMarketPrice,regularMarketChange,regularMarketTime,sharesOutstanding,bookValue&formatted=false&symbols="
  z <- fromJSON(paste(qRoot, paste(ticks, collapse=","), sep=""))
  z <- z$quoteResponse$result[,c("symbol", "regularMarketTime", "regularMarketPrice", "regularMarketChange", "longName", "sharesOutstanding", "bookValue" )]
  row.names(z) <- z$symbol
  z$symbol <- NULL
  names(z) <- c("Time", "Price", "Change", "Name", "Numeroacciones", "valorlibros")
  z$Time <- as.POSIXct(z$Time, origin = '1970-01-01 00:00:00')
  return(z)
}


ticks=Symbols

stqt=getQuote(Symbols)

stqt

#PARA CARGAR LOS ESTADOS FINANCIEROS Y FLUJOS DE EFECTIVO
url <- "https://finance.yahoo.com/quote/"
scrapy_stocks <- function(stock){
  if ("rvest" %in% installed.packages()) {
    library(rvest)
  }else{
    install.packages("rvest")
    library(rvest)
  }
  for (i in 1:length(stock)) {
    tryCatch(
      {
        
        
        # Balance Sheet
        bsheet <- paste0(url,stock[i],"/balance-sheet?p=",stock[i])
        wahis.session <- html_session(bsheet)
        p <-    wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]/div[2]') %>%
          .[[1]] %>% html_children() %>% html_nodes("div")
        suppressWarnings(Values <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 3] %>% 
                           html_text() %>% gsub(pattern = ",",replacement = "") %>% gsub(pattern = "-",replacement = 0) %>% as.numeric()) 
        Headers <- wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]') %>% 
          .[[1]] %>% html_nodes("span") %>% html_text() %>% head(5)
        Concepts <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 2] %>% html_text() %>% .[seq(2,175,4)]
        BS <- data.frame(Breakdown = Concepts[-c(1:3,11,12,22:24,32,39)], matrix(Values[!is.na(Values)], ncol = 4, byrow = T))
        names(BS) <- Headers
        temp2 <- BS
        
        
        
        # Shares Outstanding
        stock_url <- paste0(url,stock[i],"/key-statistics?p=",stock[i])
        SO <- html_session(stock_url) %>%
          html_nodes(xpath = '//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[2]/div/div[2]/div/table/tbody/tr[3]/td[2]') %>% 
          html_text() %>% gsub(pattern = "[A-z]",replacement = "") %>% as.numeric()
        temp4 <- SO
        
        assign(paste0(stock[i],'.f'),value = list(IS = temp1,BS = temp2,CF = temp3, SO = temp4),envir = parent.frame())
        
      },
      error = function(cond){
        message(stock[i], "Give error ",cond)
      }
    )
  }
}

#stock=Symbols
#scrapy_stocks(Symbols) 
#attach(AMXL.MX.f)
#AMXL.MX.f$BS
#AMXL.MX.f$IS
#AMXL.MX.f$CF
#BS

stqt$Numeroacciones


,Time,Price,Change,Name,Numeroacciones,valorlibros
,<dttm>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
AMXL.MX,2022-01-28 20:59:56,19.02,-0.01000023,"América Móvil, S.A.B. de C.V.",43554099200,3.916


[1] 43554099200

In [ ]:
TSE <- "Total stockholders' equity"
TL <- "Total Liabilities"
# dt <- colnames(BS)[2] # ÚLTIMA FECHA DISPONIBLE

dt="12/31/2020"

#K_0=BS[33,1]/1000 #Total Stockholder Equity VALOR DE LA EMPRESA
#en millones https://finance.yahoo.com/quote/AMXL.MX/balance-sheet?p=AMXL.MX
#Total stockholders' equity
#En millones
# K_0=BS[BS$Breakdown==TSE, dt]*1000/1000000
K_0=315117618*1000/1000000
#nrow(BS)
#D_0=BS[24,1]/1000 #Total Liabilities DEUDA ACTUAL
#En millones
# D_0=BS[BS$Breakdown==TL, dt]*1000/1000000
D_0=1309930609*1000/1000000

#Shares Outstanding (Sexpresa en miles de millones en la página)
#En millones
#Numeroacciones=43.55*1000000000/1000000
Numeroacciones=stqt$Numeroacciones/1000000

dt=strptime(dt, "%m/%d/%Y")
dt=format(dt, "%Y-%m")
dt
#no esta sentencia entonces mi ultima fecha fue tanto 
n=nrow(stock_prices[paste("/",dt,sep="")]) #NUMERO DE FILAS A ANALIZAR PARA EL ÚLTIMO DATO

A_0=stock_prices[paste("/",dt,sep="")][n] #valores actuales
print("precio actual de la acción")
print(A_0)

#A_0=A_0*stqt$Numeroacciones/1000000 #VALOR ACTUAL DE LAS ACCIONES VALOR DE ACCIÓN POR ACCIONES DISPONIBLES
A_0=A_0*Numeroacciones #VALOR ACTUAL DE LAS ACCIONES VALOR DE ACCIÓN POR ACCIONES DISPONIBLES
print("Ratio entre A0 y K0")
A_0/K_0

A=stock_prices[paste("/",dt,sep="")]  #VECTOR DE PRECIOS 

vol_A=sqrt(var(log(as.matrix(A[2:n])/as.matrix(A[1:(n-1)]))))*sqrt(252) #VOLATILIDAD ANUAL

#Valor del CETE

r=0.055
T=1

print("A_0")
A_0
print("K_0")
K_0
print("vol_A")
vol_A
print("Deuda")
D_0
print("r")
r
print("A/K")
A_0/K_0
T


[1] "2020-12"

[1] "precio actual de la acción"
           AMXL.MX.Close
2020-12-31         14.49
[1] "Ratio entre A0 y K0"


           AMXL.MX.Close
2020-12-31      2.002741

[1] "A_0"


           AMXL.MX.Close
2020-12-31      631098.9

...

[1] "r"


[1] 0.055

[1] "A/K"


           AMXL.MX.Close
2020-12-31      2.002741

[1] 1

In [ ]:
x=c(D_0,vol_A)

#Función para estimar vol_k y D
G = function(x) 
{
  vol_k <- x[2]
  D <- x[1]
  c(A_0-K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))+D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T))),
  A_0*vol_A-vol_k*K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))) )
}



ss=multiroot(G,x,positive = TRUE) #RESOLVER EL SISTEMA.

print("Deuda Final")
D=ss$root[1]
D
print("Volatilidad de valor de Empresa")
vol_k=ss$root[2]
vol_k
print("Incremento en Deuda") #D_T/D_0
ss$root[1]/D_0
print("Incremento en Volatilidad respecto a vol_A")#Vol_A/Vol_K
vol_A/ss$root[2]
#ss$f.root[2]/vol_A
print("Evaluacion en raíces")
G(c(D,vol_k))

#Probabilidad de incumplimiento
PD_merton=1-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))
print("PD MERTON")
PD_merton


Warning message in stode(y, times, func, parms = parms, ...):
“error during factorisation of matrix (dgefa);         singular matrix”


diagonal element is zero 
[1] 1


Warning message in stode(y, times, func, parms = parms, ...):
“steady-state not reached”


[1] "Deuda Final"


[1] 1.036751e+15

...

,AMXL.MX.Close
AMXL.MX.Close,1.775125e-09


[1] "Evaluacion en raíces"


           AMXL.MX.Close
2020-12-31  3.159813e+05
2020-12-31 -4.441405e+13

[1] "PD MERTON"


[1] 0

In [ ]:
#Sólo estimar la volatilidad dada la deuda
#Probabilidad de Incumplimiento

Vk = function(x) 
{
  vol_k <- x
  ((vol_k/vol_A)*(K_0/A_0))-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))
}

#Definir una Deuda
D=D_0
x=vol_A #semilla de la vol_k
ss=multiroot(Vk,x,positive = TRUE) #RESOLVER EL SISTEMA.
print("vol_k")
vol_k=ss$root
vol_k
print("Ratio vol_k/vol_A")
ss$root[1]/vol_A
print("valor Vk bajo solución")
Vk(vol_k)

PD_merton=1-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))

print("Deuda estimada")
D
print("Deuda sencilla")
D_0
print("Vol estimada")
vol_k
print("Vol semilla")
vol_A
print("PD Merton")
PD_merton



[1] "vol_k"


[1] 0

[1] "Ratio vol_k/vol_A"


,AMXL.MX.Close
AMXL.MX.Close,0


[1] "valor Vk bajo solución"


...

[1] 0

[1] "Vol semilla"


,AMXL.MX.Close
AMXL.MX.Close,0.2501939


[1] "PD Merton"


[1] 1

#KMV

In [ ]:
print("A_0:")
A_0
print("Valor de K_0")
K_0
print("Valor de vol_A")
vol_A
print("Valor de Deuda Actual")
D_0
print("Tasa libre de riesgo")
r
print("Temporalidad")
T


#Función para estimar vol_k y D
Gkmv = function(x) 
{
  vol_k <- x[2]
  K_0 <- x[1]
  c(A_0-K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))+D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T))),
    A_0*vol_A-vol_k*K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))) )
}

D=D_0

x=c(K_0,vol_A)

ss=multiroot(Gkmv,x,positive = TRUE) #RESOLVER EL SISTEMA.

K_e=ss$root[1]
vol_k=ss$root[2]
print("Cociente K_0/D_0")
K_e/D_0
print("Cociente vol_K/vol_A")
ss$root[2]/vol_A
print("Valor de solución en función:")
Gkmv(c(K_e,vol_k))

DI=(K_e-D)/(vol_k*K_0)
print("DI")
DI
EDF=pnorm(-DI)
print("EDF Normal")
EDF

[1] "A_0:"


           AMXL.MX.Close
2020-12-31      631098.9

[1] "Valor de K_0"


[1] 315117.6

[1] "Valor de vol_A"


...

           AMXL.MX.Close
2020-12-31      631098.9
2020-12-31      157897.1

[1] "DI"


[1] -4.600866e-07

[1] "EDF Normal"


[1] 0.5000002

##**Conclusiones**
América Móvil, S.A.B. de C.V. (AMXL.MX) tiene:

Acciones en circulación: 43554099200

K_0: 315117.618

Deuda final D: 1036751290994511

Deuda estimada de: 1309930.609

σK: 0

Volatilidad anual σA: 0.2501939

ED Normal: 0.500000183547989
